## 5.
### Generate output from test set for evaluation by TAs.

#### First train model

In [2]:
# Load libraries
import pandas as pd
import copy
import numpy as np
import timeit
import matplotlib.pyplot as plt
####
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein
####
from hw4 import neuralnetwork as NN
from hw4 import encoding as EN

In [5]:
####read in filtered negatives
with open('test/negs_filtd.txt', 'r') as file:
    neg = file.read().splitlines()
####read in positives
with open('test/pos_filtd.txt', 'r') as file:
    pos = file.read().splitlines()
####
neg_encoded,pos_encoded = EN.encode_pos_neg(neg,pos,17)

In [8]:
####define parameters
num_parts = 7
#####split up data
neg_chunks,pos_chunks = NN.chunk_kfold(num_parts,neg_encoded,pos_encoded)
#####format training data and get outputs
training_inputs,training_outputs = NN.format_trainingdata(neg_chunks,pos_chunks)
######define parameters
step_size = 0.2
iterations = 5
######start time
start = timeit.default_timer()
#####setup model
model = NN.NeuralNet(68,8,1,step_size,0.01)
#####perform k-fold cross validation
for i in range(len(training_inputs)):
    train_inputs = []
    train_outputs = []
    for j in range(len(training_inputs)):
        if i != j:
            for item in training_inputs[j]:
                train_inputs.append(item)
            for item in training_outputs[j]:
                train_outputs.append(item)
    #######
    model = NN.train_model(iterations,train_inputs,train_outputs,model)
######
total_inputs = []
total_outputs = []
for i in range(len(training_inputs)):
    for j in range(len(training_inputs[i])):
        total_inputs.append(training_inputs[i][j])
        total_outputs.append(training_outputs[i][j])
pos_predicted,pos_true,neg_predicted,neg_true = NN.call_results(total_inputs,total_outputs,model)
accuracy = NN.evaluate_accuracy(pos_predicted,pos_true,neg_predicted,neg_true)
#######stop time
stop = timeit.default_timer()
####print time
print('Time: ', stop - start)  

Time:  2.1515799439512193


#### Now put true holdout data into model for testing.

In [10]:
testset = pd.read_csv('rap1-lieb-test.txt',header=None)[0].tolist()
neg_encoded,pos_encoded = EN.encode_pos_neg(testset,testset,17)

In [18]:
with open('test/predictions.txt', 'w') as file:
    for i in range(len(pos_encoded)):
        model.prop_forward(NN.transform_string(pos_encoded[i]),NN.transform_string('1'))
        ####
        file.write(testset[i])
        file.write('\t')
        file.write(str(float(model.layer3_activation)))
        file.write('\n')